In [ ]:
# Environment variables
import os

import dotenv
import pandas as pd
from tweety import Twitter, filters

In [ ]:
app = Twitter("session")

In [ ]:
dotenv.load_dotenv()

username = os.environ.get('twitter_username')
password = os.environ.get('twitter_password')
print(username, password)

In [ ]:
app.sign_in(username=username, password=password)

In [ ]:
# Diciembre 2022 mes con mayor cantidad de robos en ese año (cifras más recientes)

# query = 'robo OR robando OR robaron OR robarle OR robado OR robada OR hurto OR huraton OR hurtado OR hurtada'
# label_text = 'robo'
# query = 'matan OR mataron OR asesinan OR asesinaron OR asesinado OR asesinada OR asesinadas OR asesinados OR femicidio OR sicariato OR acribillan OR acribillaron OR acribillado OR acribillada OR acribilladas OR acribillados OR homicidio'
# label_text = 'asesinato'
# query = 'secuestraron OR secuestro OR secuestrada OR secuestrado OR secuestradas OR secuestrados'
# label_text = 'secuestro'
# query = 'terrorismo bomba OR explosivo OR bombas OR explosivos OR bombas OR explosivos OR atentados OR atentado OR terrorista OR terroristas'
# label_text = 'terrorismo'
# query = 'extorsión OR extorsion OR extorsionaron OR extorsionado OR extorsionada OR extorsionadas OR extorsionados OR vacuna OR vacunas'
# label_text = 'extorsion'

query = ''
label_text = 'others'

# Get cursor top from file
if os.path.exists('./cursor_top.txt'):
    with open('./cursor_top.txt', 'r') as file:
        cursor_top = file.read()
else:
    cursor_top = None

all_tweets = app.search(keyword=query + ' ' + '@policiaecuador OR @fiscaliaecuador -from:@policiaecuador -from:@fiscaliaecuador lang:es since:2022-01-01 until:2023-12-31',
                        # filter_=filters.SearchFilters.Latest(),
                        pages=1,  # page = 1 => 20 tweets aprox
                        wait_time=5,  # wait_time is in seconds
                        # cursor=cursor_top, # pagination cursor of the last tweet gotten
                        )

In [ ]:
for tweet in all_tweets:
    print(tweet.text)

In [ ]:
import os

# from geotext import GeoText

date = [tweet.date for tweet in all_tweets]
text = [str(tweet.text).replace('\n', ' ') for tweet in all_tweets]
url = [tweet.url for tweet in all_tweets]
username = [tweet.author.username for tweet in all_tweets]
id = [int(tweet.id) for tweet in all_tweets]
# location = [
#     GeoText(text=tweet.text).cities for tweet in all_tweets
# ]
label = [
    label_text for _ in all_tweets
]

df = pd.DataFrame(
    data={'id': id, 'date': date, 'text': text, 'url': url, 'label': label},
)

# save to csv/json

path = f"./../datasets/non-reviewed"

if not os.path.exists(path):
    os.makedirs(path)

df.to_csv(f'./{path}/tweets - {label_text}.csv', index=False)

# save search query
with open(f'./{path}/tweets - {label_text}.txt', 'w') as f:
    f.write(all_tweets.keyword)

# df.to_json(f'./{path}/tweets.json', orient='records', force_ascii=False, date_format='iso')

# to display the full text / if n it will display n characters
pd.set_option('display.max_colwidth', None)
df

In [ ]:
# date = "2023-12-30 04:29:26+00:00"
# date = pd.to_datetime(date)
# date

test = pd.read_csv(f'./{path}/tweets - {label_text}.csv')
# test.date = test.date.apply(lambda x: pd.to_datetime(x))
test.info()